# Exercise 10 - Data frames and Statistics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import skimage.io as io

# 1. Running a basic statistical analysis
## 1.1. Introduction
There are 1000 mouse femur bones which have been measured at high resolution and a number of shape analyses run on each sample. - Phenotypical Information - Each column represents a metric which was assessed in the images - CORT_DTO__C_TH for example is the mean thickness of the cortical bone.

# 1.2 Data preparation

## 1.2.1. Load data into frames
For this example we will start with a fairly complicated dataset from a genetics analysis done at the Institute of Biomechanics, ETHZ.

In [ ]:
pheno = pd.read_csv('phenoTable.csv')
pheno.head()

Genetic Information (genoTable.csv)
Each animal has been tagged at a number of different regions of the genome (called markers: D1Mit236)
- At each marker there are 3 (actually 4) possibilities
- A is homozygous (the same from both parents) from the A strain
- B is homozygous from the B strain
- H is heterozygous (one from A, one from B)
- ‘-’ is missing or erronous measurements

In [ ]:
geno  = pd.read_csv('genoTable.csv')
geno.head(5)

## 1.2.2. [Merge data frames](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)
We want to merge the data set using the key 'ID'

In [ ]:
df = pd.merge(pheno,geno, on='ID')
df.head()

## 1.2.3. Rename a column
The key 'FEMALE' is a boolean, we want to change the key name to 'GENDER' and the contents from true/false to 'F'/'M'. Here, a [lambda function](https://realpython.com/python-lambda/) is used for the mapping when we apply an operation to each row in the GENDER column.

In [ ]:
df=df.rename(columns={"FEMALE": "GENDER"})
df['GENDER'] = df['GENDER'].apply(lambda x: 'F' if x else 'M')

df['GENDER'].sample(5)

## 1.3. Tasks

### 1.3.1 First inspection
1. Look at the histograms of the available variables in the phenotype data.

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(15,15));
pheno.hist(ax=ax,bins=50);

These are far too many variables to work with. At least for a start. We have to focus on some few e.g.
- Bone mineral density (BMD)
- Cortical bone thickness (CORT_DTO_TH)
- Cortical bone Microstructural thickness (CORT_DTO_TH_SD)

### 1.3.2 Look at the pair plot
Explore the data and correlations between various metrics by using the ‘pairplot’ plotting component. Examine different variable combinations.

In [ ]:
sns.pairplot(pheno, vars = ['BMD', 'CORT_DTO__C_TH', 'CORT_DTO__C_TH_SD']);

3. For the rest of the analysis you can connect the various components to the ‘Column Filter’ node since that is the last step in the processing
4. Use one of the T-Test to see if there is a statistically significant difference between Gender’s when examining Cortical Bone Microstructural Thickness (Mean) ```CORT_DTO__C_TH```
    - Which value is the p-value?
    - What does the p-value mean, is it significant, by what criterion?

In [ ]:
from scipy.stats import ttest_ind

# Extract the CORT_DTO__C_TH column and create a data frame for FEMALE and MALE respectively using the GENDER column
male   = df[insert your gender filters][select column to compare]
female = df[insert your gender filters][select column to compare]

ttest,pval = ttest_ind(female,male)

print("p-value={:0.4f}".format(pval))

5. Use another node from the Hypothesis Testing section to evaluate the effect on the D16Mit5 on the Lacuna Distribution Anisotropy? Is it significant?

In [ ]:
# Insert your test code here


## 1.3.3 Questions
1. In the ‘Independent Groups T-Test’ node we can run a t-test against all of the columns at the same time, why SHOULDN’T we do this?
2. If we do, how do we need to interpret this in the results
3. Is p<0.05 a sufficient signifance criteria?

## 2. Comparing two real bone samples
For this example we will compare two real cortical bone samples taken from mice.
For the purpose of the analysis and keeping the data sizes small, we will use Anders' Crazy Camera for simulating the noisy detection process. 

The assignment aims to be more integrative and you will combine a number of different lectures to get to the final answer.

### Preparing the data

In [ ]:
imgA = (0.0 < io.imread('bone_7H3A_B1.tif')).astype(float)
imgB = (0.0 < io.imread('bone_7H6A_B2.tif')).astype(float)

In [ ]:
plt.hist([imgA.ravel(), imgB.ravel()],bins=2);

This is the code to simulate a bad camera that produces bad images

In [ ]:
import numpy as numpy
import skimage.filters as filters

def camera(img,blurr=1.0,noise=0.1,illum=0.0) :

    res = filters.gaussian(img,sigma=blurr)
    res = res + np.random.normal(size=res.shape,loc=0,scale=noise)

    return res

def crappyCamera(img) :
    return camera(img,blurr=2.0,noise=0.2, illum=0.0)


In [ ]:
cimgA=crappyCamera(imgA)
cimgB=crappyCamera(imgB)

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12,7))
ax1.imshow(cimgA[100])
ax2.imshow(cimgB[100])

### 2.1 Questions
1. We want to know if there is a statistically significant difference in
    - cell volume
    - cell shape
    - cell density
  
  between the two samples given the variation in the detector
  1. which metric do we need here? 
  2. why?
2. We see in the volume comparison a very skewed representation of the data Volume/Num Pixels
    - why is this? (Hint check the segmented images)
    - What might be done to alleviate it (hint Row Filter)

### 2.2 Hints
1. Look at the kind of noise (you can peek inside the Crappy Camera) to choose the proper filter
2. Use an automated thresholding technique for finding the bone automatic-methods
3. To do this we will need to enhance the image, segment out the bone (dense) tissue, find the mask so that we can look at the cells.
4. We then need to label the cells, and analyze their volume and shape: labeling
5. Use Morphology and strongly filter parameters it might be possible to maximize the differences in the groups
6. Create a data frame with columns for sample_id, item_id and metric. Look for ideas in the previous exercises and lectures.

## 3. T-Test Simulator
This exercise (workflow named - Statistical Significance Hunter) shows the same results as we discussed in the lecture for finding p-values of significance. 

It takes a completely random stream of numbers with a mean 0.0 and tests them against a null hypothesis (that they equal 0) in small batches, you can adjust the size of the batches, the number of items and the confidence interval. The result is a pie chart showing the number of “significant” results found using the standard scientific criteria for common studies. 

### T-test for single value $\mathcal{H}_0$

In [ ]:
from scipy.stats import ttest_1samp

data= np.random.normal(size=(100,3),loc=0,scale=1.0)

tset, pval = ttest_1samp(data, 0.0) # Test if data has average = 0

print("p-values",pval)

reject = pval < 0.05
print("Reject : ", reject)
for idx,r in enumerate(reject) :
    if r :    # alpha value is 0.05 or 5%
        print("[x] we are rejecting null hypothesis")
    else:
        print("[v] we are accepting null hypothesis")

In [ ]:
def statSignificanceHunter(batches=10,samples=10) :
    testLevels=[0.05,0.01]
    data= np.random.normal(size=(samples,batches),loc=0,scale=1.0)

    tset, pval = ttest_1samp(data, 0.0)
    
    counts=[np.sum(testLevels[0]<=pval),np.sum(pval<testLevels[0])-np.sum(pval<testLevels[1]),np.sum(pval<testLevels[1])]
    return pval, counts

In [ ]:
pvals, counts = statSignificanceHunter(batches=100,samples=100)
plt.figure(figsize=(12,8))
plt.pie(counts,labels=['Accepted','0.05 Reject','0.01 Reject']); plt.legend();

### 3.1 Task
A single run is not sufficient to make a conclusion we need to repeat
Write a program that tests the random data with different batch sizes. You need:
1. Loops over batch size and number of samples (in case you want to test different sizes)
2. A loop that repeats the loops to get some statistics
3. Store the data in a data frame

In [ ]:
# Your code here

### 3.2 Questions
1. If we change the size of the chunks to the same as the number of elements in the list do we still expect to find ‘significant’ (<0.05) values? Why or why not?
2. How does comparing against the null hypothesis being 0 relate to comparing two groups?
3. How does comparing a single column compare to looking at different metrics for the same samples?
4. What is bonferroni correction (hint: wikipedia) and how could it be applied to this simulation?

Make the modification needed

## 4 Grammar of Graphics Plots 
This is a walk through demonstration using ggplot instead of matplotlib for plotting data.

### 4.1 Introduction
Making plots or graphics should be divided into separate independent components. 
- Setup is the ggplot command and the data 
- Mapping is in the aes command ```ggplot(input data frame,aes(x=name of x column,y=name of y column))+```
- Plot is the next command (geom_point, geom_smooth, geom_density, geom_histogram, geom_contour) ```geom_point()+```
- Coordinates can then be added to any type of plot (coord_equal, coord_polar, etc) 
- Scales can also be added (scale_x_log10, scale_y_sqrt, scale_color_gradientn) 
- Labels are added labs(x="x label",y="y label",title="title")

### 4.2 Tasks
1. Load the necessary libraries

In [ ]:
from plotnine import *
from plotnine.data import *

2. Load the phenoTable from the first exercise

In [ ]:
pheno = pd.read_csv('phenoTable.csv')
pheno2 = pheno[['ID','BMD','MECHANICS_STIFFNESS','CORT_DTO__C_TH','CORT_DTO__C_TH_SD']]
pheno2.head()

- Setup the input table as ```pheno``` and the map	ping with the x position mapped to BMD (Bone Mineral Density) and the y position as CT_TH_RAD (Cortical Bone Thickness)

- Create the first simple plot by adding a point representation to the plot

In [ ]:
ggplot(pheno,aes(x="BMD",y="CT_TH_RAD")) \
    + geom_point()

- Change color of the points to show if the animal is female or not (in the mapping)

In [ ]:
ggplot(pheno,aes(x="BMD",y="CT_TH_RAD",color="FEMALE"))+geom_point()

- Show the color as a discrete (factor) value instead of a number
  - First we need some data frame manipulation to change the boolean 0/1 into labels.

In [ ]:
m=pheno
m=m.rename(columns={"FEMALE": "GENDER"})
m['GENDER'] = m['GENDER'].apply(lambda x: 'F' if x else 'M')

In [ ]:
ggplot(m,aes(x="BMD",y="CT_TH_RAD",color="GENDER"))+geom_point()

- Make the plot in two facets (windows) instead of the same one

In [ ]:
ggplot(m,aes(x="BMD",y="CT_TH_RAD",color="GENDER")) \
  + geom_point() \
  + facet_wrap("GENDER")
  

For more information and tutorial read about it in: http://ggplot2.org/

## Task: Finalize the figures with decorations 
The previous plots are not publication ready, explore the different plot grammar components to add decorations 
- Plot is the next command (geom_point, geom_smooth, geom_density, geom_histogram, geom_contour) geom_point()+
- Coordinates can then be added to any type of plot (coord_equal, coord_polar, etc)
- Scales can also be added (scale_x_log10, scale_y_sqrt, scale_color_gradientn)
- Labels are added labs(x="x label",y="y label",title="title")